## Import library

In [ ]:
import os
import pickle

import numpy as np
import seaborn as sns
import pandas as pd
import scipy
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from tensorflow.keras.optimizers import RMSprop, Adam

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from IPython.core.pylabtools import figsize

## Load data

In [ ]:
# for natural sorting
import re

def tryint(s):
    try:
        return int(s)
    except:
        return s
def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]
def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)
    

# load features data
dir_path = "/tmp/elias/emotion_recognition/features"

dir_list = os.listdir(dir_path)
dir_list.sort()
dir_list.pop(0)
sort_nicely(dir_list)

dir_list = dir_list[:800]
# print(dir_list)
print(len(dir_list))

path = "/tmp/elias/emotion_recognition/features/"
features_data = []
count = 0
for file in dir_list:
    data = np.loadtxt(path+file, delimiter=',',skiprows=1, usecols=range(1,13))
    features_data.append(data)
#     print(count)
    count +=1

print("length of features data :", len(features_data))    


# load labels data    
dir_list_ = os.listdir("/tmp/elias/emotion_recognition/data")
dir_list_.sort()
dir_list_ = dir_list_[:20]
label = []

for file in dir_list_:
    dat_file = '/tmp/elias/emotion_recognition/data/' + file
    with open(dat_file, 'rb') as f:
        Channel_data =pickle.load(f,encoding='latin1')
    labels = Channel_data["labels"]
    for value in labels:
        if value[0] < 5:
            if value[1] < 5:
                label.append(0) # [0 0 0 0] V=0, A=0
            else:
                label.append(1) # [0 1 0 0] V=0, A=1
        else:
            if value[1] < 5:
                label.append(2) # [0 0 1 0] V=1, A=0
            else:
                label.append(3) # [0 0 0 0] V=1, A=1

# print("length of Arousal labels data :", len(labels_Arousal), "length of Valence labels data :", len(labels_Valence))

In [ ]:
import pandas as pd  
  
     
# list of name, degree, score 
field = ["aparna", "pankaj", "sudhir", "Geeku"] 
deg = ["MBA", "BCA", "M.Tech", "MBA"] 
scr = [90, 40, 80, 98] 
     
# dictionary of lists  
dict = {'name': nme, 'degree': deg, 'score': scr}  
       
df = pd.DataFrame(dict) 
    
# saving the dataframe 
df.to_csv('GFG.csv')

In [20]:
fname = path + "0vid_1_1.csv"
a = pd.read_csv(fname)
b = a.columns
# print(b)
b = b.to_list()
b.pop(0)
print(b)

['meanNN', 'medianNN', 'NN50', 'pNN50', 'RMSSD', 'VLF_a', 'LF_a', 'HF_a', 'Total_a', 'VLF_h', 'LF_h', 'HF_h', 'VLF_p', 'LF_p', 'HF_p', 'LF_n', 'HF_n', 'LFHF']


In [21]:
pwd


'/tmp/elias/emotion_recognition'

In [23]:
## ABORTED
## features data to_one CSV file

import csv
# field names 
fields = b 
    
# data rows of csv file 
rows = features_data

path = '/tmp/elias/emotion_recognition/features_csv/features_data.csv'
with open(path, 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(rows)

In [24]:
print("done")

done


## Assign GPU memory

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 2 GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6000)]) # limit in megabytes
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

## Split data into train and test
- what we have to do for this
 - 동영상 기준? ㅁ
 - 샘플 기준?

# train 80%, test 20%
train_data = features_data[:672]
test_data = features_data[672:]
train_data_Vlabels = labels_Valence[:672]
train_data_Alabels = labels_Arousal[:672]
test_data_Vlabels = labels_Valence[672:]
test_data_Alabels = labels_Arousal[672:]
print(len(train_data))
print(len(test_data))

In [ ]:
# train 80%, test 20%
train_data = features_data[:672]
test_data = features_data[672:]
train_data_labels = labels[:672]
test_data_labels = labels[672:]
print(len(train_data))
print(len(test_data))

## Set train and test data

In [ ]:
x_train_data = np.array(train_data)
y_train_data = np.array(train_data_labels)
x_test_data = np.array(test_data)
y_test_data = np.array(test_data_labels)
x_train_data.shape , y_train_data.shape

# # normalize data
# Train_nsamples, Train_nx, Train_ny = x_train_data.shape
# train_dataset = x_train_data.reshape((Train_nsamples,Train_nx*Train_ny))
# Test_nsamples, Test_nx, Test_ny = x_test_data.shape
# test_dataset = x_test_data.reshape((Test_nsamples,Test_nx*Test_ny))
# scaler = MinMaxScaler(feature_range=(0, 1))
# x_train_data = scaler.fit_transform(train_dataset)
# x_test_data = scaler.fit_transform(test_dataset)

# ###
# # X_train = x_train_data.reshape((x_train_data.shape[0], x_train_data.shape[1], 18))
# X_train = x_train_data.reshape((Train_nsamples, Train_nx, Train_ny))
# X_test = x_test_data.reshape((Test_nsamples, Test_nx, Test_ny))
# Y_train = y_train_data.reshape((y_train_data.shape[0], 2))
# Y_test = y_test_data.reshape((y_test_data.shape[0], 2))
# # array([[a],[b],[c]...])

# print(X_train.shape)
# print(X_test.shape)
# print(Y_train.shape)

## Build RNN, LSTM, GRU model

In [ ]:
# Set Hyperparameters of the model
learning_rate = 0.001
epochs = 200
batch_size=64

In [ ]:
X_train[0].shape

In [ ]:
# one GRU layers with three dense layers-> IT WORKS !
model = Sequential()
model.add(GRU(units=128,
#               dropout=0.25,
#               recurrent_dropout=0.25,
              return_sequences=False,
              input_shape=X_train[0].shape))

## gotta add dropout!

# Dense net
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(lr=learning_rate), metrics=['accuracy'])
model.summary()

In [ ]:
history=model.fit(X_train,
                  Y_train,
                  epochs=epochs,
                  batch_size=batch_size,
                  validation_data=(X_test,Y_test),
                  verbose=1)

In [ ]:
# plot